In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive connected.")
%cd ./drive/MyDrive/colab/tumor_classification/

Mounted at /content/drive
Google Drive connected.
/content/drive/MyDrive/colab/tumor_classification


In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d sartajbhuvaji/brain-tumor-classification-mri
!mkdir data
!unzip brain-tumor-classification-mri.zip -d data

In [ ]:
!ls data

Testing  Training


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.utils import shuffle

REBUILD_DATA = True

class BrainTumorClassification():
  IMG_SIZE = 512
  GLIOMA = "/data/Training/glioma_tumor"
  MENINGIOMA = "/data/Training/meningioma_tumor"
  PITUITARY = "/data/Training/pituitary_tumor"
  NO = "/data/Training/no_tumor"
  TESTING_GLIOMA = "/data/Testing/glioma_tumor"
  TESTING_MENINGIOMA = "/data/Testing/meningioma_tumor"
  TESTING_PITUITARY = "/data/Testing/pituitary_tumor"
  TESTING_NO = "/data/Testing/no_tumor"
  LABELS = {'glioma_tumor':0, 'meningioma_tumor': 1, 'pituitary_tumor': 2, 'no_tumor': 3}
  training_data = []
  training_images = []
  training_labels = []
  testing_data = []
  testing_images = []
  testing_labels = []

  def make_training_data(self):
    for label in self.LABELS:
      for f in tqdm(os.listdir("data/Training/"+label)):
        if "jpg" in f:
          try:
            path = "data/Training/"+os.path.join(label, f)
            img = cv2.imread(path)
            img = cv2.resize(img,(self.IMG_SIZE, self.IMG_SIZE))
            self.training_images.append(img)
            self.training_labels.append(self.LABELS[label])
          except Exception as e:
            pass
    training_images, training_labels = shuffle(self.training_images, self.training_labels, random_state=0)
    training_images = np.array(training_images)
    training_labels = np.array(training_labels)
    print(training_images.shape)
    print(training_labels.shape)
    np.save("training_images.npy", training_images)
    np.save("training_labels.npy", training_labels)

  
  def make_testing_data(self):
    for label in self.LABELS:
      for f in tqdm(os.listdir("data/Testing/"+label)):
        if "jpg" in f:
          try:
            path = "data/Testing/"+os.path.join(label, f)
            img = cv2.imread(path)
            img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
            self.testing_images.append(img)
            self.testing_labels.append(self.LABELS[label])
          except Exception as e:
            pass
    testing_images, testing_labels = shuffle(self.testing_images, self.testing_labels, random_state=0)
    testing_images = np.array(testing_images)
    testing_labels = np.array(testing_labels)
    print(testing_images.shape)
    print(testing_labels.shape)
    np.save("testing_images.npy", testing_images)
    np.save("testing_labels.npy", testing_labels)
if REBUILD_DATA:
  btclass = BrainTumorClassification()
  btclass.make_training_data()
  btclass.make_testing_data()
  

100%|██████████| 395/395 [00:03<00:00, 127.78it/s]


(2870, 512, 512, 3)
(2870,)


100%|██████████| 105/105 [00:00<00:00, 179.67it/s]


(394, 512, 512, 3)
(394,)


In [ ]:
training_images = np.load("training_images.npy", allow_pickle=True)
training_labels = np.load("training_labels.npy", allow_pickle=True)

testing_images = np.load("testing_images.npy", allow_pickle=True)
testing_labels = np.load("testing_labels.npy", allow_pickle=True)

print("Shape of training_images: ",(training_images).shape," | Shape of training_labels:",(training_labels).shape,"|| Shape of testing_images: ", (testing_images).shape ," | Shape of testing_labels:",(testing_labels).shape)

Shape of training_images:  (2870, 512, 512, 3)  | Shape of training_labels: (2870,) Shape of testing_images:  (394, 512, 512, 3)  | Shape of testing_labels: (394,)


In [ ]:
import torch
import matplotlib.pyplot as plt

X = torch.Tensor([i for i in training_images]).view(-1,512,512)
train_X = X/255.0
train_y = torch.Tensor([i for i in training_labels])

X2 = torch.Tensor([i for i in testing_images]).view(-1,512,512)
test_X = X2/255.0
test_y = torch.Tensor([i for i in testing_labels])

print(len(train_X), len(test_X))

plt.imshow(X[0], cmap="gray")
print(training_labels[0])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [ ]:
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models.mobilenet import mobilenet_v2
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss


model = mobilenet_v2(pretrained=True)
model.classifier[1] = torch.nn.Linear(in_features=model.classifier[1].in_features, out_features=4)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.MSELoss()

print(model)

In [ ]:
def train(model):
  BATCH_SIZE = 100
  EPOCHS = 10
  for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
      batch_X = train_X[i:i+BATCH_SIZE].view(-1, 3, 224, 224)
      batch_y = train_y[i:i+BATCH_SIZE]

      batch_X, batch_y = batch_X.to(device), batch_y.to(device)

      model.zero_grad()

      optimizer.zero_grad()
      outputs = model(batch_X)
      loss = loss_function(outputs, batch_y)
      loss.backward()
      optimizer.step()
      
    print(f"Epoch: {epoch}. Loss: {loss}")
train(model)

  0%|          | 0/87 [00:00<?, ?it/s]


RuntimeError: ignored